In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv0 = nn.Sequential(         
            nn.Conv2d(3, 16, 1, 1, padding = "same"),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, 1, padding = "same"),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 3, 1, 1, padding = "same"),
            nn.BatchNorm2d(3),
        )
        self.conv00 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv01 = nn.Sequential(         
            nn.Conv2d(64, 96, 1, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 96, 3, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 64, 1, 1, padding = "same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.conv02 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        identity = x
        x = self.conv0(x)
        x = identity + x
        x = self.conv1(x)
        identity = x
        x = self.conv00(x)
        x = identity + x
        x = self.conv2(x)
        identity = x
        x = self.conv01(x)
        x = identity + x
        x = self.conv3(x)
        identity = x
        x = self.conv02(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(16, 3, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv00): Sequential(
    (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), padding=

# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0447
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 3.0229
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.7869
10.344827651977539
epoch: 01 | Accuracy: 10.34 Loss: 80.7037

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.5818
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.2977
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.5764
25.208086013793945
epoch: 02 | Accuracy: 25.21 Loss: 66.7295

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 1.9644
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 1.9361
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.7259
44.11415100097656
epoch: 03 | Accuracy: 44.11 Loss: 50.2903

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.4750
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.6157
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.9060
61.35552978515625
epoch: 04 | Accuracy: 61.36 Loss: 34.3450

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 0.8793
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.5989
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 0.9364
72.7705078125
epoch: 05 | Accuracy: 72.77 Loss: 23.6364

Epoch: [6/30]	|	Step: [9/27

In [7]:
print(Accy_list)

[10.344827651977539, 25.208086013793945, 44.11415100097656, 61.35552978515625, 72.7705078125, 83.23424530029297, 88.58501434326172, 90.01189422607422, 92.27110290527344, 95.7193832397461, 99.04875183105469, 99.04875183105469, 99.88109588623047, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 98.09750366210938, 91.31985473632812, 94.17359924316406, 97.62187957763672, 96.6706314086914, 99.28656005859375, 99.40547180175781, 99.28656005859375]


In [8]:
print(Loss_list)

[80.70367383956909, 66.72948944568634, 50.29031765460968, 34.34502148628235, 23.636419594287872, 15.094307228922844, 10.855210304260254, 8.103011310100555, 7.299825191497803, 3.767831467092037, 1.6308974046260118, 1.1209277994930744, 0.6592594408430159, 0.3545506256632507, 0.21061947685666382, 0.15470729139633477, 0.11861868295818567, 0.1225861320272088, 0.09435031679458916, 0.07725379732437432, 0.06830775109119713, 0.10106621484737843, 1.793769568321295, 7.326347654685378, 4.856211110949516, 2.6408623661845922, 2.7230022586882114, 0.7619187431409955, 0.8232821496203542, 0.8086671438068151]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 88.11 %, Loss: 15.7463 
